In [1]:
from utils import AverageMeter, load_inputs, get_png_names, get_ys
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torchvision import transforms, utils, models
from model import RecurrentAttention
import torch.nn.functional as F
import matplotlib.pyplot as plt
from config import get_config
from trainer import Trainer
from PIL import Image
from tqdm import tqdm
import pandas as pd
import torchvision
import shutil
import pickle
import torch
import utils
import json
import time

import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID" 
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

In [2]:
m = open("../../pooling/data/migration_data.json",)
mig_data = json.load(m)
m.close()
mig_data = pd.DataFrame.from_dict(mig_data, orient = 'index').reset_index()
mig_data.columns = ['muni_id', 'num_migrants']
q = 4
mig_data['class'] = pd.qcut(mig_data['num_migrants'], q = q, labels = [i for i in range(q)])
mig_data

,muni_id,num_migrants,class
0,484001001,42055.0,3
1,484001002,4017.0,3
2,484001003,11992.0,3
3,484001004,762.0,2
4,484001005,7551.0,3
...,...,...,...
2326,484032049,2487.0,3
2327,484032050,2024.0,3
2328,484032051,3084.0,3
2329,484032052,2919.0,3


In [3]:
mig_data['class'].value_counts()

0    583
1    583
2    583
3    582
Name: class, dtype: int64

In [4]:
image_names = get_png_names("../../attn/data/MEX/")

y_class, y_mig = get_ys(image_names, mig_data)

In [5]:
import matplotlib.pyplot as plt
import torchvision

In [6]:
import random

train_num = int(25 * .70)

train_indices = random.sample(range(0, 25), train_num)
val_indices = [i for i in range(0, 25) if i not in train_indices]

In [7]:
import numpy as np
from utils import plot_images

import torch
from torchvision import datasets
from torchvision import transforms
from torch.utils.data.sampler import SubsetRandomSampler

In [13]:
class miniConv(torch.nn.Module):
    
    def __init__(self, resnet):
        
        super().__init__()
        
        self.conv1_miniConv = resnet.conv1.to(device)
        self.bn1_miniConv = resnet.bn1.to(device)
        self.relu_miniConv = resnet.relu.to(device)
        self.maxpool_miniConv = resnet.maxpool.to(device)
        self.layer1_miniConv = resnet.layer1.to(device)
        self.layer2_miniConv = resnet.layer2.to(device)
        self.adp_pool_miniConv = torch.nn.AdaptiveAvgPool2d((1, 1)).to(device)        
        D_in = 1 * 128 * 1 * 1
        h_g = 128
        self.fc1 = torch.nn.Linear(D_in, h_g)#.to('cuda:1')
        
    def forward(self, phi):
        
        phi = self.conv1_miniConv(phi)
        phi = self.bn1_miniConv(phi)
        phi = self.relu_miniConv(phi)
        phi = self.maxpool_miniConv(phi)
        phi = self.layer1_miniConv(phi)
        phi = self.layer2_miniConv(phi)
        phi = self.adp_pool_miniConv(phi)
        phi = phi.flatten(start_dim = 1)  # Keep a batch_size = num_glimpses for predictions at each scale
        phi_out = F.relu(self.fc1(phi)) # feed phi to respective fc layer
        return phi

In [14]:
device = "cuda"

In [24]:
resnet = torchvision.models.resnet18()
miniConv_model = miniConv(resnet = resnet).to(device)

# params = []
# for n,p in miniConv_model.named_parameters():
#     print(n)
#     params.append(n)

In [25]:
# checkpoint = torch.load("./ckpt_og/ram_4_50x50_0.75_model_best.pth.tar")
# checkpoint = checkpoint["model_state"]
# checkpoint.keys()

In [26]:
# [p for p in list(checkpoint.keys()) if "_miniConv" in p]

In [27]:
# matching_keys2

In [28]:
checkpoint = torch.load("./ckpt/ram_4_50x50_0.75_model_best.pth.tar")
checkpoint = checkpoint["model_state"]

matching_keys = [p for p in list(checkpoint.keys()) if "_miniConv" in p]
matching_keys2 = [p for p in list(checkpoint.keys()) if p == 'sensor.fc1.weight' or p == 'sensor.fc1.bias']
[matching_keys2.append(k) for k in matching_keys]

weights = {}
for mk in matching_keys2:
    miniConv_model_key = mk.replace("sensor.", "")
    weights[miniConv_model_key] = checkpoint[mk]
    
miniConv_model.load_state_dict(weights)

<All keys matched successfully>

In [29]:
# g = open("./locations_extract.json")
g = open("./sample4.json")
glimpses = json.load(g)
g.close()

In [30]:
# glimpses

In [39]:
features_dict = {}

for im_name, patches in glimpses.items():
        
    muni_id = im_name.split("/")[5]
    print(muni_id)
    image = load_inputs(im_name)
        
    cur_features_dict = {}
    
    for glimpseID, coords in patches.items():
        
        if "_scale0" in glimpseID:
        
#             print(glimpseID)

            from_x, to_x, from_y, to_y = int(coords["from_x"]), int(coords["to_x"]), int(coords["from_y"]), int(coords["to_y"])
            cur_patch = image[:, :, from_x:to_x, from_y:to_y]

    #         print(.shape)
    #         plt.imshow(cur_patch[0].permute(1,2,0))
    #         plt.savefig(f"{muni_id}_{glimpseID}.png")

            features = miniConv_model(cur_patch.to(device))[0].detach().cpu().numpy()
            features = [str(f) for f in features]
            cur_features_dict[glimpseID] = features

    features_dict[im_name] = cur_features_dict
    
#     print("\n")

484001008
484004004
484004001
484005010
484001005
484001007
484001006
484001001
484004005
484005001
484005004
484005008
484001002
484004006
484001004
484005013
484001003
484005003
484005018
484005019
484005016
484005015
484005014
484005017
484005021


In [40]:
features_dict

{'../../attn/data/MEX/484001008/pngs/484001008_2010_all_box484001008_MAY.png': {'glimpse0_scale0': ['0.096422866',
   '0.0027287405',
   '0.11080639',
   '0.32729325',
   '0.070314154',
   '0.19492292',
   '0.83566093',
   '0.019036938',
   '0.008171494',
   '0.27063555',
   '0.05743108',
   '0.08809224',
   '0.006225421',
   '0.06878377',
   '0.15383844',
   '1.1397144',
   '0.051428676',
   '0.0986988',
   '0.06741561',
   '0.015703212',
   '0.061081044',
   '0.038290262',
   '0.0042982567',
   '0.07312761',
   '0.08837533',
   '0.58182174',
   '0.058232795',
   '0.08228388',
   '0.3910283',
   '0.13006431',
   '0.055829596',
   '0.1044313',
   '0.02527074',
   '0.15345852',
   '1.0399945',
   '0.17451715',
   '0.01078706',
   '0.3295676',
   '0.075044066',
   '0.10439972',
   '0.2418631',
   '0.091715805',
   '0.097932436',
   '0.054153807',
   '0.029884305',
   '0.41405192',
   '0.027728705',
   '0.30685103',
   '0.049333107',
   '0.22647397',
   '0.043654446',
   '0.11334238',
   

In [60]:
features_dict

{'../../attn/data/MEX/484001008/pngs/484001008_2010_all_box484001008_MAY.png': {'glimpse0': ['-44.88124',
   '-39.92961',
   '221.50955',
   '-34.240314',
   '-33.833168',
   '-31.659412',
   '-36.741894',
   '-9.048515',
   '-37.73855',
   '-31.947834',
   '-4.4747167',
   '-27.681953',
   '-34.825977',
   '-86.80033',
   '-17.73956',
   '-35.69273',
   '-16.610807',
   '-8.130311',
   '-38.442776',
   '-14.94148',
   '-37.167625',
   '-36.84255',
   '-15.255919',
   '-7.2356253',
   '-13.500128',
   '-38.32555',
   '-10.28603',
   '-32.936977',
   '-37.20842',
   '-38.209892',
   '-21.926714',
   '-22.633228',
   '-16.519236',
   '-33.14192',
   '-61.73199',
   '-34.47637',
   '-36.944942',
   '-25.73253',
   '-7.091854',
   '-32.857883',
   '-9.381997',
   '-42.37945',
   '-34.4636',
   '-35.739',
   '-21.124607',
   '-38.819756',
   '-38.16056',
   '-37.079376',
   '-27.251635',
   '-33.787388',
   '-57.498173',
   '-26.584455',
   '-40.772465',
   '439.22916',
   '-22.900564',
   

In [73]:
# cur_features_dict["glimpse0"]

In [74]:
with open("sample_features.json", "w") as outfile: 
    json.dump(features_dict, outfile)